In [1]:
import clip
import tome

import torch

/home/lyx/anaconda3/envs/tome/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)

In [3]:
# Set this to be whatever device you want to benchmark on
# If you don't have a GPU, you can use "cpu" but you probably want to set the # runs to be lower
device = "cuda:0"
runs = 50
batch_size = 256  # Lower this if you don't have that much memory
input_size = [3,224,224]

In [4]:
classes = ["dog", "cat", "wolf", "people", "stone", "grass"]

text_dict = torch.cat([clip.tokenize(c) for c in classes]).to(device)

In [5]:
# Baseline benchmark
model = model.float()
baseline_throughput = tome.utils.benchmark(
    model,
    device=device,
    verbose=True,
    runs=runs,
    batch_size=batch_size,
    input_size=input_size,
    text = text_dict
)

Benchmarking: 100%|██████████| 50/50 [03:41<00:00,  4.44s/it]

Throughput: 57.81 im/s


In [6]:
from tome.patch.clip import patch_openclip
patch_openclip(model, 4)

In [7]:
text_dict.dtype

torch.int32

In [8]:
tome_throughput = tome.utils.benchmark(
    model,
    device=device,
    verbose=True,
    runs=runs,
    batch_size=batch_size,
    input_size=input_size,
    text = text_dict
)

Benchmarking:   0%|          | 0/50 [00:00<?, ?it/s]/home/lyx/code/tome/tome/merge.py:75: UserWarning: scatter_reduce() is in beta and the API may change at any time. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1664766666830/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1550.)
  dst = dst.scatter_reduce(-2, dst_idx.expand(n, r, c), src, reduce=mode)
Benchmarking: 100%|██████████| 50/50 [03:03<00:00,  3.67s/it]

Throughput: 69.69 im/s


In [9]:
model.r = 16
tome_throughput = tome.utils.benchmark(
    model,
    device=device,
    verbose=True,
    runs=runs,
    batch_size=batch_size,
    input_size=input_size,
    text = text_dict
)

Benchmarking: 100%|██████████| 50/50 [03:02<00:00,  3.65s/it]

Throughput: 69.83 im/s


In [10]:
patch_openclip(model, 16)

AttributeError: 'ToMeAttention' object has no attribute 'embed_dim'